# 3D GOL with text visualization (cut at grid z=0)

In [1]:
import numpy as np
import random

class GameOfLife3D:
    """
    Implements a 3D Cellular Automaton with 3 states (0: Dead, 1: Type A, 2: Type B)
    using periodic boundary conditions and a defined rule set.
    """

    def __init__(self, size_l, size_w, size_h):
        """
        Initializes the 3D Game of Life grid.

        Args:
            size_l (int): Length of the grid (x-dimension).
            size_w (int): Width of the grid (y-dimension).
            size_h (int): Height of the grid (z-dimension).
        """
        if not (isinstance(size_l, int) and size_l > 0 and
                  isinstance(size_w, int) and size_w > 0 and
                  isinstance(size_h, int) and size_h > 0):
            raise ValueError("Grid dimensions must be positive integers.")

        self.size_l = size_l
        self.size_w = size_w
        self.size_h = size_h
        self.grid = np.zeros((size_l, size_w, size_h), dtype=int)
        print(f"Initialized grid with shape: {self.grid.shape}")

    def randomize(self, density_a=0.1, density_b=0.1):
        """
        Fills the grid randomly with Type A and Type B cells based on densities.

        Args:
            density_a (float): Approximate density of Type A cells (0 to 1).
            density_b (float): Approximate density of Type B cells (0 to 1).
        """
        if not (0 <= density_a <= 1 and 0 <= density_b <= 1):
             raise ValueError("Densities must be between 0 and 1.")
        if density_a + density_b > 1:
            raise ValueError("Sum of densities cannot exceed 1.")

        total_cells = self.size_l * self.size_w * self.size_h
        num_a = int(total_cells * density_a)
        num_b = int(total_cells * density_b)

        # Create a flat list of indices
        indices = list(range(total_cells))
        random.shuffle(indices)

        # Assign states based on shuffled indices
        flat_grid = np.zeros(total_cells, dtype=int)
        for i in range(num_a):
            flat_grid[indices[i]] = 1 # Type A
        for i in range(num_a, num_a + num_b):
            flat_grid[indices[i]] = 2 # Type B

        # Reshape the flat array back into the 3D grid
        self.grid = flat_grid.reshape((self.size_l, self.size_w, self.size_h))
        print(f"Randomized grid with approx {density_a*100:.1f}% Type A, {density_b*100:.1f}% Type B.")

    def _count_neighbors(self, x, y, z):
        """
        Counts the number of Type A and Type B neighbors for a given cell
        using periodic boundary conditions.

        Args:
            x (int): x-coordinate of the cell.
            y (int): y-coordinate of the cell.
            z (int): z-coordinate of the cell.

        Returns:
            tuple: (count_a, count_b), the number of Type A and Type B neighbors.
        """
        count_a = 0
        count_b = 0
        for i in [-1, 0, 1]:
            for j in [-1, 0, 1]:
                for k in [-1, 0, 1]:
                    # Skip the cell itself
                    if i == 0 and j == 0 and k == 0:
                        continue

                    # Calculate neighbor coordinates with periodic boundaries
                    nx = (x + i) % self.size_l
                    ny = (y + j) % self.size_w
                    nz = (z + k) % self.size_h

                    neighbor_state = self.grid[nx, ny, nz]
                    if neighbor_state == 1:
                        count_a += 1
                    elif neighbor_state == 2:
                        count_b += 1
        return count_a, count_b

    def _apply_rules(self, current_state, count_a, count_b):
        """
        Applies the placeholder game rules to determine the next state of a cell.

        Args:
            current_state (int): The current state of the cell (0, 1, or 2).
            count_a (int): Number of Type A neighbors.
            count_b (int): Number of Type B neighbors.

        Returns:
            int: The next state of the cell (0, 1, or 2).
        """
        next_state = current_state
        live_neighbors = count_a + count_b

        if current_state == 0: # Dead cell
            if count_a == 3 and count_b == 0:
                next_state = 1 # Birth of Type A
            elif count_b == 3 and count_a == 0:
                next_state = 2 # Birth of Type B
            # else: stays Dead (already initialized)
        elif current_state == 1: # Type A cell
            if live_neighbors < 2 or live_neighbors > 3:
                next_state = 0 # Dies (underpopulation or overpopulation)
            # else: stays Type A (Survival)
        elif current_state == 2: # Type B cell
            if live_neighbors < 3 or live_neighbors > 4:
                next_state = 0 # Dies (underpopulation or overpopulation)
            # else: stays Type B (Survival)

        return next_state

    def step(self):
        """
        Advances the game state by one time step.
        Calculates the next state for all cells based on the current state,
        then updates the grid.
        """
        # Create a new grid to store the next state
        new_grid = np.zeros_like(self.grid)

        # Iterate through each cell in the grid
        for x in range(self.size_l):
            for y in range(self.size_w):
                for z in range(self.size_h):
                    current_state = self.grid[x, y, z]
                    count_a, count_b = self._count_neighbors(x, y, z)
                    new_grid[x, y, z] = self._apply_rules(current_state, count_a, count_b)

        # Update the grid with the new state
        self.grid = new_grid

    def get_grid(self):
        """Returns the current state of the grid."""
        return self.grid.copy() # Return a copy to prevent external modification

    def set_grid(self, new_grid_state):
        """
        Sets the grid to a specific state. Useful for initialization or testing.

        Args:
            new_grid_state (np.ndarray): A 3D numpy array matching the grid dimensions
                                         and containing states (0, 1, 2).
        """
        if not isinstance(new_grid_state, np.ndarray):
             raise TypeError("Input must be a numpy array.")
        if new_grid_state.shape != (self.size_l, self.size_w, self.size_h):
            raise ValueError(f"Input array shape mismatch. Expected {self.grid.shape}, got {new_grid_state.shape}")
        if not np.all(np.isin(new_grid_state, [0, 1, 2])):
             raise ValueError("Grid contains invalid state values. Only 0, 1, 2 are allowed.")
        self.grid = new_grid_state.astype(int) # Ensure dtype is int

    def get_state(self, x, y, z):
        """
        Gets the state of a specific cell.

        Args:
            x (int): x-coordinate.
            y (int): y-coordinate.
            z (int): z-coordinate.

        Returns:
            int: State of the cell (0, 1, or 2).
        """
        if not (0 <= x < self.size_l and 0 <= y < self.size_w and 0 <= z < self.size_h):
            raise IndexError("Coordinates out of grid bounds.")
        return self.grid[x, y, z]

    def display_slice(self, z_index=0, slice_axis='z'):
        """
        Prints a 2D slice of the 3D grid for basic visualization.

        Args:
            z_index (int): The index of the slice along the z-axis (default 0).
            slice_axis (str): The axis along which to slice ('x', 'y', or 'z'). Default 'z'.
        """
        print(f"--- Grid Slice ({slice_axis}={z_index}) ---")
        try:
            if slice_axis == 'z':
                if not (0 <= z_index < self.size_h): raise IndexError("z_index out of bounds")
                slice_2d = self.grid[:, :, z_index]
            elif slice_axis == 'y':
                 if not (0 <= z_index < self.size_w): raise IndexError("y_index out of bounds")
                 slice_2d = self.grid[:, z_index, :]
            elif slice_axis == 'x':
                 if not (0 <= z_index < self.size_l): raise IndexError("x_index out of bounds")
                 slice_2d = self.grid[z_index, :, :]
            else:
                raise ValueError("slice_axis must be 'x', 'y', or 'z'")

            for row in slice_2d:
                print(" ".join(map(str, row)))
        except IndexError as e:
            print(f"Error displaying slice: {e}")
        print("----------------------")


In [2]:
if __name__ == "__main__":
    # Create a 10x10x5 grid
    game = GameOfLife3D(size_l=10, size_w=10, size_h=5)

    # Randomize the initial state
    game.randomize(density_a=0.15, density_b=0.05)

    # Display the initial state (slice z=0)
    print("Initial State:")
    game.display_slice(z_index=0)
    game.display_slice(z_index=1)


    # Run for a few steps
    num_steps = 5
    for i in range(num_steps):
        game.step()
        print(f"\nAfter Step {i+1}:")
        game.display_slice(z_index=0)
        # game.display_slice(z_index=1) # Optionally display other slices

    # Get the final grid state
    final_grid = game.get_grid()
    print(f"\nFinal grid shape: {final_grid.shape}")
    unique, counts = np.unique(final_grid, return_counts=True)
    print(f"Final state counts: {dict(zip(unique, counts))}")

Initialized grid with shape: (10, 10, 5)
Randomized grid with approx 15.0% Type A, 5.0% Type B.
Initial State:
--- Grid Slice (z=0) ---
0 0 0 0 0 1 0 0 0 0
0 0 0 0 0 0 2 0 0 0
1 0 0 2 0 0 0 0 0 0
2 0 0 0 0 0 1 0 0 0
0 0 0 0 0 1 0 0 2 0
0 0 0 2 0 0 0 0 0 0
1 0 0 0 1 2 0 0 0 0
0 0 0 1 1 0 0 0 0 0
0 1 0 0 0 1 0 0 0 0
0 0 0 0 0 0 0 1 0 0
----------------------
--- Grid Slice (z=1) ---
0 0 0 0 0 1 0 1 0 0
1 2 1 0 0 0 0 0 0 0
0 1 1 0 1 0 0 0 0 0
0 2 0 0 0 0 0 0 0 0
1 0 0 0 0 0 0 0 0 0
0 0 0 0 0 1 0 1 0 0
1 0 0 0 1 0 0 0 0 0
0 2 0 0 0 0 0 0 0 0
0 0 1 0 0 0 0 0 0 1
1 0 1 0 0 1 0 0 0 0
----------------------

After Step 1:
--- Grid Slice (z=0) ---
0 0 0 0 0 0 0 0 1 0
0 0 0 0 0 0 0 0 1 0
0 0 0 2 0 0 0 0 0 0
0 0 0 0 0 1 1 0 0 0
0 0 0 0 0 1 0 0 0 0
0 0 0 2 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
----------------------

After Step 2:
--- Grid Slice (z=0) ---
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 1 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 

# GOL with 3D interaction visualization

In [3]:
import numpy as np
import random

class GameOfLife3D:
    """
    Implements a 3D Cellular Automaton with 3 states (0: Dead, 1: Type A, 2: Type B)
    using periodic boundary conditions and a defined rule set.
    """

    def __init__(self, size_l, size_w, size_h):
        """
        Initializes the 3D Game of Life grid.

        Args:
            size_l (int): Length of the grid (x-dimension).
            size_w (int): Width of the grid (y-dimension).
            size_h (int): Height of the grid (z-dimension).
        """
        if not (isinstance(size_l, int) and size_l > 0 and
                  isinstance(size_w, int) and size_w > 0 and
                  isinstance(size_h, int) and size_h > 0):
            raise ValueError("Grid dimensions must be positive integers.")

        self.size_l = size_l
        self.size_w = size_w
        self.size_h = size_h
        self.grid = np.zeros((size_l, size_w, size_h), dtype=int)
        print(f"Initialized grid with shape: {self.grid.shape}")

    def randomize(self, density_a=0.1, density_b=0.1):
        """
        Fills the grid randomly with Type A and Type B cells based on densities.

        Args:
            density_a (float): Approximate density of Type A cells (0 to 1).
            density_b (float): Approximate density of Type B cells (0 to 1).
        """
        if not (0 <= density_a <= 1 and 0 <= density_b <= 1):
             raise ValueError("Densities must be between 0 and 1.")
        if density_a + density_b > 1:
            raise ValueError("Sum of densities cannot exceed 1.")

        total_cells = self.size_l * self.size_w * self.size_h
        num_a = int(total_cells * density_a)
        num_b = int(total_cells * density_b)

        # Create a flat list of indices
        indices = list(range(total_cells))
        random.shuffle(indices)

        # Assign states based on shuffled indices
        flat_grid = np.zeros(total_cells, dtype=int)
        for i in range(num_a):
            flat_grid[indices[i]] = 1 # Type A
        for i in range(num_a, num_a + num_b):
            flat_grid[indices[i]] = 2 # Type B

        # Reshape the flat array back into the 3D grid
        self.grid = flat_grid.reshape((self.size_l, self.size_w, self.size_h))
        print(f"Randomized grid with approx {density_a*100:.1f}% Type A, {density_b*100:.1f}% Type B.")

    def _count_neighbors(self, x, y, z):
        """
        Counts the number of Type A and Type B neighbors for a given cell
        using periodic boundary conditions.

        Args:
            x (int): x-coordinate of the cell.
            y (int): y-coordinate of the cell.
            z (int): z-coordinate of the cell.

        Returns:
            tuple: (count_a, count_b), the number of Type A and Type B neighbors.
        """
        count_a = 0
        count_b = 0
        for i in [-1, 0, 1]:
            for j in [-1, 0, 1]:
                for k in [-1, 0, 1]:
                    # Skip the cell itself
                    if i == 0 and j == 0 and k == 0:
                        continue

                    # Calculate neighbor coordinates with periodic boundaries
                    nx = (x + i) % self.size_l
                    ny = (y + j) % self.size_w
                    nz = (z + k) % self.size_h

                    neighbor_state = self.grid[nx, ny, nz]
                    if neighbor_state == 1:
                        count_a += 1
                    elif neighbor_state == 2:
                        count_b += 1
        return count_a, count_b

    def _apply_rules(self, current_state, count_a, count_b):
        """
        Applies the placeholder game rules to determine the next state of a cell.

        Args:
            current_state (int): The current state of the cell (0, 1, or 2).
            count_a (int): Number of Type A neighbors.
            count_b (int): Number of Type B neighbors.

        Returns:
            int: The next state of the cell (0, 1, or 2).
        """
        next_state = current_state
        live_neighbors = count_a + count_b

        if current_state == 0: # Dead cell
            if count_a == 3 and count_b == 0:
                next_state = 1 # Birth of Type A
            elif count_b == 3 and count_a == 0:
                next_state = 2 # Birth of Type B
            # else: stays Dead (already initialized)
        elif current_state == 1: # Type A cell
            if live_neighbors < 2 or live_neighbors > 3:
                next_state = 0 # Dies (underpopulation or overpopulation)
            # else: stays Type A (Survival)
        elif current_state == 2: # Type B cell
            if live_neighbors < 3 or live_neighbors > 4:
                next_state = 0 # Dies (underpopulation or overpopulation)
            # else: stays Type B (Survival)

        return next_state

    def step(self):
        """
        Advances the game state by one time step.
        Calculates the next state for all cells based on the current state,
        then updates the grid.
        """
        # Create a new grid to store the next state
        new_grid = np.zeros_like(self.grid)

        # Iterate through each cell in the grid
        for x in range(self.size_l):
            for y in range(self.size_w):
                for z in range(self.size_h):
                    current_state = self.grid[x, y, z]
                    count_a, count_b = self._count_neighbors(x, y, z)
                    new_grid[x, y, z] = self._apply_rules(current_state, count_a, count_b)

        # Update the grid with the new state
        self.grid = new_grid

    def get_grid(self):
        """Returns the current state of the grid."""
        return self.grid.copy() # Return a copy to prevent external modification

    def set_grid(self, new_grid_state):
        """
        Sets the grid to a specific state. Useful for initialization or testing.

        Args:
            new_grid_state (np.ndarray): A 3D numpy array matching the grid dimensions
                                         and containing states (0, 1, 2).
        """
        if not isinstance(new_grid_state, np.ndarray):
             raise TypeError("Input must be a numpy array.")
        if new_grid_state.shape != (self.size_l, self.size_w, self.size_h):
            raise ValueError(f"Input array shape mismatch. Expected {self.grid.shape}, got {new_grid_state.shape}")
        if not np.all(np.isin(new_grid_state, [0, 1, 2])):
             raise ValueError("Grid contains invalid state values. Only 0, 1, 2 are allowed.")
        self.grid = new_grid_state.astype(int) # Ensure dtype is int

    def get_state(self, x, y, z):
        """
        Gets the state of a specific cell.

        Args:
            x (int): x-coordinate.
            y (int): y-coordinate.
            z (int): z-coordinate.

        Returns:
            int: State of the cell (0, 1, or 2).
        """
        if not (0 <= x < self.size_l and 0 <= y < self.size_w and 0 <= z < self.size_h):
            raise IndexError("Coordinates out of grid bounds.")
        return self.grid[x, y, z]

    def display_slice(self, z_index=0, slice_axis='z'):
        """
        Prints a 2D slice of the 3D grid for basic visualization.

        Args:
            z_index (int): The index of the slice along the z-axis (default 0).
            slice_axis (str): The axis along which to slice ('x', 'y', or 'z'). Default 'z'.
        """
        print(f"--- Grid Slice ({slice_axis}={z_index}) ---")
        try:
            if slice_axis == 'z':
                if not (0 <= z_index < self.size_h): raise IndexError("z_index out of bounds")
                slice_2d = self.grid[:, :, z_index]
            elif slice_axis == 'y':
                 if not (0 <= z_index < self.size_w): raise IndexError("y_index out of bounds")
                 slice_2d = self.grid[:, z_index, :]
            elif slice_axis == 'x':
                 if not (0 <= z_index < self.size_l): raise IndexError("x_index out of bounds")
                 slice_2d = self.grid[z_index, :, :]
            else:
                raise ValueError("slice_axis must be 'x', 'y', or 'z'")

            for row in slice_2d:
                print(" ".join(map(str, row)))
        except IndexError as e:
            print(f"Error displaying slice: {e}")
        print("----------------------")

In [4]:
# --- Example Usage ---
if __name__ == "__main__":
    # Create a 10x10x5 grid
    game = GameOfLife3D(size_l=10, size_w=10, size_h=5)

    # Randomize the initial state
    game.randomize(density_a=0.15, density_b=0.05)

    # Display the initial state (slice z=0)
    print("Initial State:")
    game.display_slice(z_index=0)
    game.display_slice(z_index=1)


    # Run for a few steps
    num_steps = 5
    for i in range(num_steps):
        game.step()
        print(f"\nAfter Step {i+1}:")
        game.display_slice(z_index=0)
        # game.display_slice(z_index=1) # Optionally display other slices

    # Get the final grid state
    final_grid = game.get_grid()
    print(f"\nFinal grid shape: {final_grid.shape}")
    unique, counts = np.unique(final_grid, return_counts=True)
    print(f"Final state counts: {dict(zip(unique, counts))}")

Initialized grid with shape: (10, 10, 5)
Randomized grid with approx 15.0% Type A, 5.0% Type B.
Initial State:
--- Grid Slice (z=0) ---
0 0 0 0 0 1 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 1 1 0 0
0 0 0 0 1 1 1 0 0 0
1 0 0 2 0 0 0 0 0 1
0 0 0 0 0 0 0 1 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 2 0 0 1 1 0 0 2
0 0 0 0 0 0 1 0 0 1
----------------------
--- Grid Slice (z=1) ---
0 0 1 0 0 1 0 0 0 0
0 0 0 1 2 1 0 0 0 0
0 0 0 0 0 0 0 0 2 0
1 0 0 0 2 0 1 0 1 0
0 0 0 0 0 2 0 2 0 0
0 0 0 0 0 1 0 0 0 0
0 1 0 2 0 0 0 0 0 1
1 1 1 0 0 2 1 0 0 0
0 0 0 0 0 1 2 0 0 0
0 0 2 1 0 0 0 1 1 0
----------------------

After Step 1:
--- Grid Slice (z=0) ---
1 0 0 0 0 0 0 0 0 1
0 0 0 0 0 0 0 0 0 0
1 1 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
1 0 0 2 0 0 0 0 0 0
0 0 0 0 0 0 0 1 0 0
0 0 0 0 0 0 0 0 0 1
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 2
0 0 0 0 0 0 0 0 0 0
----------------------

After Step 2:
--- Grid Slice (z=0) ---
1 1 0 0 0 0 0 0 1 1
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 